In [1]:
import pandas as pd
from os import listdir
import os
import csv
import tensorflow as tf

from tqdm import tqdm
from tqdm import trange

c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

In [2]:
filepath = ['./dogs/n02085936-maltese_dog', './dogs/n02113712-miniature_poodle']

filelist = []
for i in range(len(filepath)):
    for x in listdir(filepath[i]):
        fpath = str(filepath[i])
        filelist.append(fpath +'/' + x + ',' + str(i) + '\n')
    with open('./dogs/label_0.csv', 'w', encoding='utf-8', newline='') as csvfile:
        csvfile.writelines(filelist)

In [85]:
def test_speed(ds, test_size=5000, warmup=None, print_top_n=0):
    it = ds.make_one_shot_iterator()
    next_dp = it.get_next()
    try:
        with tf.Session() as sess:
            if warmup:
                for _ in trange(warmup):
                    sess.run(next_dp)

            with tqdm(total=test_size, leave=True, smoothing=0.2) as pbar:
                for _ in range(test_size):
                    if _ < print_top_n:
                        dp = sess.run(next_dp)
                        print('shape: {}'.format([item.shape for item in dp if hasattr(item, 'shape')]))

                        print('data: {}'.format(dp))
                    else:
                        sess.run(next_dp)
                    pbar.update()
    except tf.errors.OutOfRangeError as e:
        print('OutOfRange')

In [95]:
tf.reset_default_graph()

df = pd.read_csv('./dogs/label_0.csv', names=['filename', 'label'])
filename = df['filename'].values

ds = tf.data.Dataset.from_tensor_slices((filename, df['label'].values))
ds = ds.repeat().shuffle(300)

def preprocessing(filename, label):
    file = tf.read_file(filename)
    image = tf.image.decode_image(file, channels=3)
    image.set_shape([None,None,3])
    image = tf.image.resize_images(image, [64,64])
    image = tf.image.per_image_standardization(image)  # normalization
    
    return (image, tf.one_hot(label, 2))

ds = ds.map(preprocessing, num_parallel_calls=2)   # num_parailel_calls=3 : cpu 활용 증가 시킨
ds = ds.batch(128)
ds = ds.prefetch(10)  # 

test_speed(ds, test_size=1000, print_top_n=3)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


  0%|                                                                                 | 1/1000 [00:00<04:59,  3.34it/s]

shape: [(128, 64, 64, 3), (128, 2)]
data: (array([[[[ 0.89699763,  1.1553404 ,  1.4592732 ],
         [ 1.0708092 ,  1.06701   ,  1.1914325 ],
         [ 1.071759  ,  1.0565623 ,  1.1591396 ],
         ...,
         [-1.671234  , -1.6864307 , -1.5952508 ],
         [-1.7016274 , -1.7225227 , -1.6693345 ],
         [-0.54003435,  0.07732903,  0.2197975 ]],

        [[ 1.0318679 ,  1.13492   ,  1.401811  ],
         [ 1.0314226 ,  0.903201  ,  0.9449917 ],
         [ 1.0326395 ,  0.9377199 ,  0.9883556 ],
         ...,
         [-1.5263318 , -1.477002  , -1.2120106 ],
         [-1.529834  , -1.4919019 , -1.4187087 ],
         [-0.6264356 ,  0.06248857,  0.14194442]],

        [[ 1.1496418 ,  1.2408216 ,  1.4535745 ],
         [ 0.91557795,  0.77560264,  0.7920459 ],
         [ 0.8141285 ,  0.6431663 ,  0.6614498 ],
         ...,
         [-1.5230075 , -1.4765865 , -1.2723223 ],
         [-1.6189957 , -1.5265099 , -1.4005439 ],
         [-1.7641947 , -1.4632895 , -1.369557  ]],

        .

  0%|▏                                                                                | 2/1000 [00:00<04:41,  3.54it/s]

data: (array([[[[ 1.38629997e+00,  6.55298710e-01, -2.33756766e-01],
         [ 1.50113630e+00,  7.89891839e-01, -1.90538794e-01],
         [ 1.61597264e+00,  9.17076170e-01, -1.34972826e-01],
         ...,
         [ 8.61510217e-01,  2.68806547e-01, -4.81951416e-01],
         [ 7.07160294e-01,  1.93483800e-01, -5.37517369e-01],
         [ 6.31837547e-01,  1.18161038e-01, -6.12840176e-01]],

        [[ 1.42581356e+00,  7.14569092e-01, -2.33756766e-01],
         [ 1.53694546e+00,  8.25701058e-01, -1.89728454e-01],
         [ 1.64807737e+00,  9.56589758e-01, -1.10276833e-01],
         ...,
         [ 9.01023805e-01,  3.08320135e-01, -4.42437857e-01],
         [ 7.46673882e-01,  2.32997373e-01, -4.98003811e-01],
         [ 6.86072230e-01,  1.72395736e-01, -5.58605433e-01]],

        [[ 1.49928403e+00,  7.88039684e-01, -2.28200167e-01],
         [ 1.59436357e+00,  9.02876019e-01, -1.87065914e-01],
         [ 1.69291604e+00,  1.00142837e+00, -8.02557766e-02],
         ...,
         [ 9.4852

  0%|▏                                                                                | 3/1000 [00:00<04:26,  3.74it/s]

data: (array([[[[ 1.751885  ,  1.5074732 ,  0.30763355],
         [ 1.8768684 ,  1.6546758 ,  0.2770821 ],
         [ 1.8685361 ,  1.6324565 ,  0.25208542],
         ...,
         [-1.9809495 , -1.9809495 , -1.9809495 ],
         [-1.9809495 , -1.9809495 , -1.9809495 ],
         [-1.9809495 , -1.9809495 , -1.9809495 ]],

        [[ 1.9011706 ,  1.6216941 ,  0.06947093],
         [ 1.741166  ,  1.5210565 , -0.34316924],
         [ 1.916186  ,  1.6796725 ,  0.06903697],
         ...,
         [-1.9809495 , -1.9809495 , -1.9809495 ],
         [-1.9809495 , -1.9809495 , -1.9809495 ],
         [-1.9809495 , -1.9809495 , -1.9809495 ]],

        [[ 2.0567055 ,  1.7234167 ,  0.2222283 ],
         [ 1.778878  ,  1.5173243 , -0.402775  ],
         [ 1.9752059 ,  1.7192506 ,  0.02911174],
         ...,
         [-1.9809495 , -1.9809495 , -1.9809495 ],
         [-1.9809495 , -1.9809495 , -1.9809495 ],
         [-1.9809495 , -1.9809495 , -1.9809495 ]],

        ...,

        [[-0.8783191 , -0.79881

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:38<00:00,  4.58it/s]


```
it = ds.make_one_shot_iterator()
n = it.get_next()
```

```
with tf.Session() as sess:
    for _ in range(3):
        print(sess.run(n))
```

# Tensorflow-Slim 사용해 보기
- Slim에는 다양한 모델이 있다. 
- Slim을 활용하기 위해서는 slim 폴더가 있는 경로 설정이 필요하다.
- 프로프트에서 하기의 경로 설정이 필요하다.

```
echo %CONDA_PREFIX%

cd %CONDA_PREFIX%
mkdir .\etc\conda\activate.d
mkdir .\etc\conda\deactivate.d
type NUL > .\etc\conda\activate.d\env_vars.bat
type NUL > .\etc\conda\deactivate.d\env_vars.bat
```

- notepad가 열리면 경로 설정한다. 

```
notepad .\etc\conda\activate.d\env_vars.bat
set PYTHONPATH= Slim 경로
```

```
notepad .\etc\conda\deactivate.d\env_vars.bat
set PYTHONPATH=
```

In [3]:
from nets import cifarnet
mynet = cifarnet.cifarnet


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

